In [46]:
import GetOldTweets3 as got3
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time

I will extract top tweets for the past 10 years. 100 top tweets will be extracted each month on the specified topic for past 10 years

In [47]:

def get_tweets(topic, end_date, years=10):
    columns = ['id', 'url', 'author', 'retweets', 'favorites', 'mentions', 'hashtags', 'geo', 'time', 'text']
    text_query = topic
    
    end_date_datetime = datetime.strptime(end_date, '%Y-%m-%d')

    
    date_range = [end_date_datetime]
    total_months = 12 * years
    
    prior_date = end_date_datetime
    
    while total_months > 1:
        prior_date = prior_date - relativedelta(months=1)
        date_range.append(prior_date)
        total_months -= 1
        
    
    date_range_string = [twitter_date.strftime('%Y-%m-%d') for twitter_date in date_range][::-1]
    
    
    
    compiled_tweets_list = []
    
    for i in range(len(date_range_string)-1):
        tweetCriteria = got3.manager\
                .TweetCriteria()\
                .setQuerySearch(text_query)\
                .setLang('en')\
                .setSince(date_range_string[i])\
                .setUntil(date_range_string[i+1])\
                .setTopTweets(True)\
                .setMaxTweets(100)

        tweets = got3.manager.TweetManager.getTweets(tweetCriteria)

        text_tweets = [[tweet.id, tweet.permalink, tweet.username, tweet.retweets, 
                tweet.favorites, tweet.mentions, tweet.hashtags, tweet.geo, 
                tweet.date, tweet.text] for tweet in tweets]

        sample_tweets = pd.DataFrame(text_tweets, columns=columns)
        compiled_tweets_list.append(sample_tweets)
        time.sleep(30)
        
    return pd.concat(compiled_tweets_list)
    

In [49]:
tweets = get_tweets("immigration", "2019-12-01")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



An error occured during an HTTP request: HTTP Error 429: Too Many Requests
Try to open in browser: https://twitter.com/search?q=immigration%20since%3A2010-01-01%20until%3A2010-02-01&src=typd
Traceback (most recent call last):
  File "D:\Anaconda_Navigator\envs\computational_content_analysis\lib\site-packages\GetOldTweets3\manager\TweetManager.py", line 343, in getJsonResponse
    response = opener.open(url)
  File "D:\Anaconda_Navigator\envs\computational_content_analysis\lib\urllib\request.py", line 532, in open
    response = meth(req, response)
  File "D:\Anaconda_Navigator\envs\computational_content_analysis\lib\urllib\request.py", line 642, in http_response
    'http', request, response, code, msg, hdrs)
  File "D:\Anaconda_Navigator\envs\computational_content_analysis\lib\urllib\request.py", line 570, in error
    return self._call_chain(*args)
  File "D:\Anaconda_Navigator\envs\computational_content_analysis\lib\urllib\request.py", line 504, in _call_chain
    result = func(*arg

SystemExit: 

In [45]:
tweets

,id,url,author,retweets,favorites,mentions,hashtags,geo,time,text
